In [2]:
import os
import zipfile
import gdown
from pathlib import Path
from dataclasses import dataclass
from Chest_Cancer_Classification.constants import *
from Chest_Cancer_Classification import logger
from Chest_Cancer_Classification.utils.common import read_yaml, create_directories, get_size

In [3]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
class ConfigurationManager:
    """
    This class is responsible for managing the configuration of the project.
    It reads the configuration and parameters from YAML files and provides methods to access them.
    
    Attributes:
        config (dict): Configuration settings.
        params (dict): Parameters settings.
    """
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        This method is responsible for setting up the data ingestion configuration.
        It creates the necessary directories and prepares the configuration for data ingestion.

        Returns:
            DataIngestionConfig: The data ingestion configuration object.
        """
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )
        return data_ingestion_config

In [5]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, str(zip_download_dir))
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            logger.error(f"Error occurred while downloading the file: {e}")
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        logger.info(f"Extracting file {self.config.local_data_file} into directory {unzip_path}")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2025-05-03 11:08:57,036 - INFO - common - YAML file /home/letruongzzio/chest-cancer-classification/config/config.yaml loaded successfully.
2025-05-03 11:08:57,039 - INFO - common - YAML file /home/letruongzzio/chest-cancer-classification/params.yaml loaded successfully.
2025-05-03 11:08:57,039 - INFO - common - Directory artifacts created successfully.
2025-05-03 11:08:57,040 - INFO - common - Directory artifacts/data_ingestion created successfully.
2025-05-03 11:08:57,041 - INFO - 3733540353 - Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=3b756d0f-bcb4-45a7-8053-dba42f3a3485
To: /home/letruongzzio/chest-cancer-classification/research/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:02<00:00, 23.4MB/s]

2025-05-03 11:09:04,501 - INFO - 3733540353 - Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip
2025-05-03 11:09:04,501 - INFO - 3733540353 - Extracting file artifacts/data_ingestion/data.zip into directory artifacts/data_ingestion
